# Coins Tutorial


Imagine that it is the end of the calendar year at the United States Mint.
The Mint keeps an inventory of the various minerals used to produce the coins that are put into circulation, and it wants to use up the minerals on hand before retooling for next year's coins.

The Mint produces several different types of coins, each with a different composition. The table below shows the make-up of each coin type.


| |Penny|Nickel|Dime|Quarter|Dollar|
| -------- | ------- | -------- | ------- | -------- | ------- |
|Copper|0.06g|3.8g|2.1g|5.2g|7.2g|
|Nickel||1.2g|0.2g|0.5g|0.2g|
|Zinc|2.4g||||0.5g|
|Manganese|||||0.3g|

In order to formulate this as an optimization problem, we'll need to do three things.

- First, we'll need to define the decision variables. The goal of the optimization is to choose values for these variables.
- Second, we'll define a linear objective function. This is the function we'd like to minimize (or maximize).
- Third, we'll define the linear constraints.

The Gurobi Optimizer will consider all assignments of values to decision variables that satisfy the specified linear constraints, and return one that optimizes the stated objective function.

The variables in this problem are quite straightforward. The solver will need to decide how many of each coin to produce. It is convenient to give the decision variables meaningful names. In this case, we'll call the variables Pennies, Nickels, Dimes, Quarters, and Dollars. We'll also introduce variables that capture the quantities of the various minerals actually used by the solution. We'll call them Cu, Ni, Zi, and Mn.

Recall that the objective of our optimization problem is to maximize the total dollar value of the coins produced. Each penny produced is worth 0.01 dollars, each nickel is worth 0.05 dollars, etc. This gives the following linear objective:

> maximize: 0.01 Pennies + 0.05 Nickels + 0.1 Dimes + 0.25 Quarters + 1 Dollars

The constraints of this model come from the fact that producing a coin consumes certain quantities of the available minerals, and the supplies of those minerals are limited. We'll capture these relationships in two parts. First, we'll create an equation for each mineral that captures the amount of that mineral that is consumed. For copper, that equation would be:

> Cu = 0.06 Pennies + 3.8 Nickels + 2.1 Dimes + 5.2 Quarters + 7.2 Dollars


The coefficients for this equation come from the earlier coin specification table: one penny uses 0.06g of copper, one nickel uses 3.8g, etc.

The model must also capture the available quantities of each mineral. If we have 1000 grams of copper available, then the constraint would be:

> Cu <= 1000

For our example, we'll assume we have 1000 grams of copper and 50 grams of the other minerals.

There is actually one other set of constraints that must be captured in order for our model to accurately reflect the physical realities of our problem. While a dime is worth 10 cents, half of a dime isn't worth 5 cents. The variables that capture the number of each coin produced must take integer values.

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
try:
    # Create a new model
    m = gp.Model("coins1")

    # Create variables
    pennies = m.addVar(vtype=GRB.CONTINUOUS, name="pennies")
    nickels = m.addVar(vtype=GRB.CONTINUOUS, name="nickels")
    dimes = m.addVar(vtype=GRB.CONTINUOUS, name="dimes")
    quarters = m.addVar(vtype=GRB.CONTINUOUS, name="quarters")
    dollars = m.addVar(vtype=GRB.CONTINUOUS, name="dollars")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1
y 0
z 1
Obj: 3
